<a href="https://colab.research.google.com/github/hurileborjigin/pytorch_notes/blob/main/CNN_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convolutional Neural Network

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
num_epochs = 4
batch_size = 4
lr = 0.001

In [6]:
# Dataset has PILImages of range [0, 1]
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [7]:
# Prepare the dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:03<00:00, 42.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [13]:
# Create the model class
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))

    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)





    return x

In [14]:
model = ConvNet().to(device)

In [15]:
# losses and optimizer

model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [16]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
      print(f"Epoch: [{epoch+1}/{num_epochs}], Step: [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")

print("Finished Training")

Epoch: [1/4], Step: [2000/12500], Loss: 2.2810
Epoch: [1/4], Step: [4000/12500], Loss: 2.2855
Epoch: [1/4], Step: [6000/12500], Loss: 2.3706
Epoch: [1/4], Step: [8000/12500], Loss: 2.0713
Epoch: [1/4], Step: [10000/12500], Loss: 2.1340
Epoch: [1/4], Step: [12000/12500], Loss: 3.0935
Epoch: [2/4], Step: [2000/12500], Loss: 2.1923
Epoch: [2/4], Step: [4000/12500], Loss: 2.7113
Epoch: [2/4], Step: [6000/12500], Loss: 1.5993
Epoch: [2/4], Step: [8000/12500], Loss: 2.2415
Epoch: [2/4], Step: [10000/12500], Loss: 2.4870
Epoch: [2/4], Step: [12000/12500], Loss: 1.3243
Epoch: [3/4], Step: [2000/12500], Loss: 2.6085
Epoch: [3/4], Step: [4000/12500], Loss: 2.9751
Epoch: [3/4], Step: [6000/12500], Loss: 2.7669
Epoch: [3/4], Step: [8000/12500], Loss: 2.7454
Epoch: [3/4], Step: [10000/12500], Loss: 1.2627
Epoch: [3/4], Step: [12000/12500], Loss: 1.3845
Epoch: [4/4], Step: [2000/12500], Loss: 1.1672
Epoch: [4/4], Step: [4000/12500], Loss: 1.8931
Epoch: [4/4], Step: [6000/12500], Loss: 2.2162
Epoch: 

In [17]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]

  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs, 1) # torch.max returns value, index
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f"Accuracy of the network: {acc} %")
  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f"Accuracy of {classes[i]}: {acc} %")





Accuracy of the network: 47.35 %
Accuracy of plane: 50.3 %
Accuracy of car: 62.6 %
Accuracy of bird: 27.3 %
Accuracy of cat: 29.3 %
Accuracy of deer: 30.0 %
Accuracy of dog: 49.7 %
Accuracy of frog: 51.9 %
Accuracy of horse: 63.2 %
Accuracy of ship: 60.8 %
Accuracy of truck: 48.4 %
